In [1]:
# KNN + Boarder Detection + Normal Direction Constrains

# 1) KNN search: several candidates of corresponding points
# 2) Boarder Detection + Normal Direction constrains => remove some bad correspondences

In [2]:
import math
import pickle
import os
import time
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors

## SMPL model
data_root   = '/home/yan/Data2/NOMO_Project_Summary/'
root_folder = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_model  = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl'
out_folder  = root_folder + 'optimized_mesh/dataset01/'
scan_folder =  data_root + 'Data/TC2_scan/ori_scan/re_name/dataset01/female/'

# pre-process initial params
# param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'
param_folder = None

# load SMPL model
model = load_model(smpl_model)
scan_list = os.listdir(scan_folder)

deg2rad = 1.0 / 180 * math.pi
bnds = ((-20*deg2rad, 10*deg2rad), (-15*deg2rad,  15*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, 10*deg2rad), (-15*deg2rad,  15*deg2rad), (30*deg2rad,   90*deg2rad),
        (0*deg2rad,   30*deg2rad), (-30*deg2rad, 0*deg2rad), (0*deg2rad,    15*deg2rad),  
        (0*deg2rad,   30*deg2rad), (0*deg2rad,  30*deg2rad), (-15*deg2rad,   10*deg2rad),  
        (-10*deg2rad,  10*deg2rad), (-10*deg2rad,  10*deg2rad), (-5*deg2rad,    15*deg2rad), 
        (-10*deg2rad,   10*deg2rad), (-10*deg2rad,  10*deg2rad), (-15*deg2rad,   5*deg2rad),
        (-5*deg2rad,   15*deg2rad), (-5*deg2rad,  15*deg2rad), (-10*deg2rad,    10*deg2rad),
        (-15*deg2rad,  5*deg2rad), (-15*deg2rad, 5*deg2rad), (-15*deg2rad,   5*deg2rad), 
        (-15*deg2rad,  5*deg2rad), (-5*deg2rad,  15*deg2rad), (-5*deg2rad,    15*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [3]:
def loadTarget(scanFile):
    ''' load NOMO3D scans, return Vertices and Faces'''
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV, targetF = [], []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            elif CC[0] == 'f':
                targetF.append([int(CC[1]), int(CC[2]), int(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    targetF = np.array(targetF, dtype=int) - 1 # .obj file, is 0-based, started from 0
    
    return targetV, targetF

def loadInitParam(paramFile):
    '''load initial parameters, including shape and pose from the previous experiments'''
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def normalize_v3(arr):
    '''Normalize a numpy array of 3 component vectors shape=(n, 3)'''
    lens = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def getNormals(V, F):
    ''' Creating a normal per triangle, is done by the cross product of two vectors on this triangle.
    Creating smooth normals, i.e. one normal per vertex is done 
    by averaging the normal of all the triangles one vertex is part of.
    
    https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy'''
    
    norm = np.zeros(V.shape, dtype=V.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = V[F] 
    # Calculate the normal for all the triangles, 
    # by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle 
    n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
    n = normalize_v3(n)
    # Multiple triangles would then contribute to every vertex
    norm[F[:, 0]] += n
    norm[F[:, 1]] += n
    norm[F[:, 2]] += n
    norm = normalize_v3(norm)
    
    return norm

def getBoarderV(Faces):
    '''the boarder of the mesh are those edges which belong to only one polygon'''
    boundV = []
    edges01 = np.array([Faces[:, 0], Faces[:, 1]], dtype=int).T
    edges02 = np.array([Faces[:, 0], Faces[:, 2]], dtype=int).T
    edges03 = np.array([Faces[:, 1], Faces[:, 2]], dtype=int).T
    edge = np.sort(np.concatenate((edges01, edges02, edges03), axis=0))
    
    unique_e, n_counts = np.unique(edge, axis=0, return_counts=True)
    boarder_edge = unique_e[n_counts==1, :]
    boundV = np.unique(boarder_edge.flatten())
    
    return boundV

def getDist(dist_arr, idx_arr, K, modelN):
    ''' get the weighted mean distance from the model to scan
            dist_arr : 6890*K
            idx_arr  : 6890*K
                K    : the number of KNN neighbour, K > 1
            modelN   : 6890*3, the normals of modelV
    '''
    angle_arr = np.zeros((6890, K), dtype=float)
    for kk in range(K):
        correspN = targetNorm[idx_arr[:, kk], :]
        norm_dot = np.einsum('ij,ij->i', modelN, correspN) # dot
        angle_arr[:, kk] = np.absolute(np.arccos(np.clip(norm_dot, -1.0, 1.0))) # 6890*1
    # find the smallest angle in each row
    col_idx = np.argmin(angle_arr, axis=1)          # (6890,) = [0 1 1 0 0 0 1 1 ...]
    idx     = np.choose(col_idx, idx_arr.T)
    angle   = np.choose(col_idx, angle_arr.T)
    dist    = np.choose(col_idx, dist_arr.T)
    
    W = np.ones((6890,), dtype=float)                # 0 for bad correspondences, otherwise 1  
    W[np.nonzero(dist > 0.1)]             = 0        # 10 cm
    W[np.nonzero(angle > math.pi / 4)]    = 0        # 45 degree
    W[np.nonzero(np.isin(idx, boarderV))] = 0        # boarder vertices

    return np.sum(np.multiply(dist, W)) / np.count_nonzero(W)

def meshDist(param):
    '''the distance between the SMPL model and the target scan'''
    
    model.betas[:]    = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18]  = param[12:27]
    
    modelF = model.f # 0-based, started from 0
    modelV = model.r - np.mean(model.r, 0)
    modelNorm = getNormals(modelV, modelF)
    
    N_neigh = 5                                       # KNN correspondence search,  find the cloest corresponding
    t_dist, t_idx = kdtree.query(modelV, k=N_neigh)   # t_dist: 6890*K, t_idx: 6890*K
    
    return getDist(t_dist, t_idx, N_neigh, modelNorm) * 1000 # [mm]

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

for scan in scan_list:
    scan_name = scan[:-4]

    targetV, targetF = loadTarget(scan_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    boarderV   = getBoarderV(targetF)                             # the vertices on the boarder edges
    targetNorm = getNormals(targetV, targetF)                   # the normal for every target vertices
    
    kdtree = scipy.spatial.cKDTree(targetV, leafsize=6890)      # KNN search
    
#     paramFile = param_folder + scan_name + '.pkl'               # None for female samples                       
    paramFile = None
    init_pose, init_betas = loadInitParam(paramFile)            # female does not have the pre-param
    
    init_param        = np.zeros(37)                            # 27 part pose parameters + 10 shape param
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    start_time = time.time()
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, 
                   options={'gtol': 1e-4, 'maxiter': 25, 'disp': True})
    end_time = time.time()
    print "eclipse time [second] : ", end_time - start_time
    
    optimal_param = res.x
    print scan_name, " : Objective Func : ", res.fun
    print " shape : ", optimal_param[27:]
    
    #------------------------------------------------------------------------------------
    # save
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/female02/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/female_param02/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)


eclipse time [second] :  3386.77465796
female_0135  : Objective Func :  22.592530252665807
 shape :  [-0.5224854  -0.29103481 -0.1435413   0.07678227 -0.03056727 -0.13080397
 -0.04000579  0.03792338  0.02504569 -0.01916783]
eclipse time [second] :  2338.53067493
female_0012  : Objective Func :  23.61511997189351
 shape :  [-0.00861984  0.05042703 -0.03836647  0.01512461  0.03810771 -0.17608413
  0.0403019   0.01708336 -0.00422417 -0.01683662]
eclipse time [second] :  767.211741924
female_0168  : Objective Func :  19.359603880234584
 shape :  [-0.79934578 -0.10646426 -0.29265769  0.1147879   0.0956382  -0.24040706
 -0.21779074  0.01500896  0.03777089  0.00157627]
eclipse time [second] :  2328.03626609
female_0164  : Objective Func :  21.330973766025707
 shape :  [-0.37114184  0.40543215 -0.13793153 -0.04054479  0.04688116 -0.181643
  0.023542   -0.01617135  0.01711611  0.01607192]
eclipse time [second] :  2689.61662507
female_0094  : Objective Func :  23.19290442813478
 shape :  [ 0.012

eclipse time [second] :  1281.6816411
female_0051  : Objective Func :  26.132517552256918
 shape :  [ 0.06577052 -0.1000843  -0.0516449   0.09214084 -0.04009121 -0.0438982
 -0.01327776  0.07213216  0.01417105  0.00847725]
eclipse time [second] :  2746.88200283
female_0137  : Objective Func :  29.47754209540914
 shape :  [-1.57173996  0.20506262 -0.29356286  0.11316155  0.18288541 -0.76780066
 -0.33210783 -0.18826906  0.07309463  0.04058622]
eclipse time [second] :  1060.47623301
female_0010  : Objective Func :  26.698025767616848
 shape :  [-0.51753212  0.13622499 -0.00218844 -0.02210363  0.03773162 -0.1637333
 -0.0422847  -0.00979427 -0.02109885  0.06176747]
eclipse time [second] :  2138.89818311
female_0133  : Objective Func :  20.590114511405726
 shape :  [ 0.03691366  0.06213373 -0.03711969  0.0106586   0.00424926 -0.06620517
  0.00413553  0.01423691  0.00135915 -0.00513866]
eclipse time [second] :  1694.340518
female_0065  : Objective Func :  25.62703401573836
 shape :  [-0.329838

eclipse time [second] :  1910.80416298
female_0004  : Objective Func :  15.527899942074654
 shape :  [ 0.16403081 -0.11859155 -0.0242276   0.00759233  0.03633007 -0.08902861
  0.06672375  0.02600146 -0.0155847  -0.01412938]
eclipse time [second] :  2947.28894401
female_0066  : Objective Func :  19.251179971860953
 shape :  [-0.00843179  0.22129294 -0.08615724  0.03410905  0.06394095 -0.11763594
 -0.0229397   0.01427056  0.01861523  0.00632497]
eclipse time [second] :  3900.36455703
female_0109  : Objective Func :  21.277365556563478
 shape :  [-1.53021485  1.36593111 -0.75262252 -0.1994949   0.41140143 -1.14866907
  0.1110263  -0.03259424  0.04011733  0.05638852]
eclipse time [second] :  2900.3304801
female_0061  : Objective Func :  22.138432923151303
 shape :  [-0.86256119  0.87637122 -0.28741556 -0.02353288  0.14136003 -0.61562237
 -0.08815188  0.00318176 -0.00215552 -0.014549  ]
eclipse time [second] :  1883.36071205
female_0056  : Objective Func :  19.20134656043617
 shape :  [-0.1

eclipse time [second] :  3241.30627894
female_0020  : Objective Func :  21.840008863560275
 shape :  [-0.28284409  0.03923222 -0.03084344 -0.01052491  0.05893432 -0.23394995
  0.0207704  -0.01970514  0.01046688 -0.02619923]
eclipse time [second] :  3418.90395594
female_0124  : Objective Func :  21.27197818526507
 shape :  [-0.31366143 -0.17299745 -0.00844821  0.01920867  0.00167382 -0.06078598
 -0.0068906  -0.00606076  0.00430214  0.00549352]
eclipse time [second] :  1052.77923608
female_0144  : Objective Func :  20.05334980021377
 shape :  [-0.06902297 -0.27435093  0.09126799  0.05237323 -0.01557763 -0.17339554
 -0.02387192 -0.01372889  0.00222889  0.00583458]
eclipse time [second] :  3635.62310696
female_0022  : Objective Func :  23.969136883462095
 shape :  [-0.14583176  0.01513143 -0.09956479  0.05846839 -0.02034211 -0.18720306
 -0.01431243  0.05364194  0.01107041 -0.00927234]
eclipse time [second] :  2293.8628149
female_0091  : Objective Func :  20.898440152106883
 shape :  [ 0.03

eclipse time [second] :  1214.41667199
female_0104  : Objective Func :  17.539287459674476
 shape :  [-1.68832353  0.02785172 -0.37218378 -0.09068028  0.07586566 -0.99033011
 -0.28320725  0.01224152  0.03543266 -0.07544942]
eclipse time [second] :  1900.4686389
female_0021  : Objective Func :  28.648692418661085
 shape :  [-2.16921934 -0.08120101 -0.13391917  0.06361463  0.00341198 -1.09865469
  0.2203505  -0.04161488 -0.04169606  0.25042792]
eclipse time [second] :  2948.81515598
female_0139  : Objective Func :  16.918562668573635
 shape :  [-0.12468602  1.10643543 -0.37686952 -0.15036608  0.04910821 -0.32360486
  0.03864049  0.06994568  0.01392003 -0.0323522 ]
eclipse time [second] :  2066.39639783
female_0017  : Objective Func :  26.879166686319458
 shape :  [-3.          0.1648773   0.19905494 -0.15981006  0.27920987 -0.59572679
  0.33297709 -0.25948179  0.00861195  0.25469162]
eclipse time [second] :  1135.83383012
female_0067  : Objective Func :  17.713565998250132
 shape :  [-0.

In [5]:
print "done"

done
